In [1]:
%pip install lxml
%pip install requests

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
### Trustpilot data scraping module 
### Author: Hakim Khalafi

## Imports

import math
import csv
import time
import json
import requests
import lxml.html as html

In [3]:
## Configurations

# Trustpilot review page
basePage = 'http://www.trustpilot.com/review/'
reviewSite = 'www.fwd.com.sg'
reviewPage = basePage + reviewSite

# Data file to save to
datafile = 'datafwd.csv'

# Trustpilot default 
resultsPerPage = 20 

print('Scraper set for ' + reviewPage + ' - saving result to ' + datafile)

Scraper set for http://www.trustpilot.com/review/www.fwd.com.sg - saving result to datafwd.csv


In [11]:
## Count amount of pages to scrape

# Get page, skipping HTTPS as it gives certificate errors
page = requests.get(reviewPage, verify=False)
tree = html.fromstring(page.content)

# Total amount of ratings
ratingCount = tree.xpath('//span[@class="typography_typography__QgicV typography_h2__wAVpO typography_weight-medium__UNMDK typography_fontstyle-normal__kHyN3 styles_reviewCount__wGBxK"]')
ratingCount = int(ratingCount[0].text.replace(',',''))

# Amount of chunks to consider for displaying processing output 
# For ex. 10 means output progress for every 10th of the data
tot_chunks = 20

# Throttling to avoid spamming page with requests
# With sleepTime seconds between every page request
throttle = True
sleepTime = 1

# Total pages to scrape
pages = math.ceil(ratingCount / resultsPerPage)
print('Found total of ' + str(pages) + ' pages to scrape')

Found total of 1121 pages to scrape


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.trustpilot.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
## Main scraping section

with open(datafile, 'w', newline='', encoding='utf8') as csvfile:
    
    # Tab delimited to allow for special characters
    datawriter = csv.writer(csvfile, delimiter='\t')
    print('Processing..')
    for i in range(1,pages+1):
        
        # Sleep if throttle enabled
        if(throttle): time.sleep(sleepTime)

        page = requests.get(reviewPage + '?page=' + str(i))
        tree = html.fromstring(page.content)
        
        # Each item below scrapes a pages review titles, bodies and ratings
        script_bodies = tree.xpath("//script[starts-with(@data-initial-state, 'review-info')]")
        for idx,elem in enumerate(script_bodies):
            curr_item = json.loads(elem.text_content())

            # Progress counting, outputs for every processed chunk
            reviewNumber = idx + 20*(i-1) + 1
            chunk = int(ratingCount / tot_chunks)
            if(reviewNumber % chunk == 0): 
                print('Processed ' + str(reviewNumber) + '/'  + str(ratingCount) + ' ratings')
            
            title = curr_item["reviewHeader"]
            body = curr_item["reviewBody"]
            rating = curr_item["stars"]
            
            print([title,body,rating])
            datawriter.writerow([title,body,rating])
            
    print('Processed ' + str(ratingCount) + '/' + str(ratingCount) + ' ratings.. Finished!')

Processing..


KeyboardInterrupt: 